In [1]:
DEVICE="cuda"

In [44]:
import gc
import numpy as np
import pandas as pd
import sklearn.metrics
from PIL import Image, ImageFile
import torch, torch.nn
import torch.nn.functional as F
import torchvision.transforms.v2 as tf
from transformers import AutoModelForImageClassification
from datasets import load_from_disk
from fastprogress import progress_bar as pb
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:

transforms = {key:{} for key in [224, 256]}
datasets = {}

def train_transform_224(ex):
    if "pixel_values" in ex:
        ex["pixel_values"]=[transforms[224]["train"](image) for image in ex["pixel_values"]]
    return ex
def test_transform_224(ex):
    if "pixel_values" in ex:
        ex["pixel_values"]=[transforms[224]["test"](image) for image in ex["pixel_values"]]
    return ex
def train_transform_256(ex):
    if "pixel_values" in ex:
        ex["pixel_values"]=[transforms[256]["train"](image) for image in ex["pixel_values"]]
    return ex
def test_transform_256(ex):
    if "pixel_values" in ex:
        ex["pixel_values"]=[transforms[256]["test"](image) for image in ex["pixel_values"]]
    return ex

for size in [224, 256]:
    transforms[size]["train"] = tf.Compose([
        tf.ToImage(),
        tf.ToDtype(torch.float32, scale=True),
        tf.RandomRotation(degrees=20),
        tf.RandomHorizontalFlip(),
        tf.RandomResizedCrop(size=(size,size), scale=(.8,1.0), antialias=True),
        tf.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    transforms[size]["test"] = tf.Compose([
        tf.ToImage(),
        tf.ToDtype(torch.float32, scale=True),  # Normalize expects float input
        tf.Resize(size=(size,size)),
        tf.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    dataset=load_from_disk("./data/dataset/")
    dataset["train"].set_format("torch")
    dataset["test"].set_format("torch")
    dataset["train"]=dataset["train"].rename_column("image","pixel_values")
    dataset["test"]=dataset["test"].rename_column("image","pixel_values")
    datasets[size]=dataset

datasets[224]["train"].set_transform(train_transform_224)
datasets[224]["test"].set_transform(train_transform_224)
datasets[256]["train"].set_transform(train_transform_256)
datasets[256]["test"].set_transform(train_transform_256)


In [4]:
dataset=datasets[256] #default

In [5]:
datasets[224]["train"][0]["pixel_values"].shape

torch.Size([3, 224, 224])

In [6]:
datasets[256]["train"][0]["pixel_values"].shape

torch.Size([3, 256, 256])

In [7]:
#gather training data statistics

In [8]:
y_train=torch.tensor(dataset["train"]["class"])
y_test=torch.tensor(dataset["test"]["class"])

In [9]:
y_total=torch.concat([y_train,y_test])

In [10]:
labels=dataset["test"].features["class"].names

In [11]:
model_list={
    "mirluvams/swinv2-base-patch4-window16-256-popocatepetl":256,
    "mirluvams/swinv2-tiny-patch4-window16-256-popocatepetl":256,
    "mirluvams/beit-base-patch16-224-popocatepetl":224,
    "mirluvams/vit-base-patch16-224-popocatepetl":224,
}
y_pred = {}
model=None

In [12]:
BS=8
for model_name, size in model_list.items():
    model=AutoModelForImageClassification.from_pretrained(model_name).to(DEVICE)
    print("Evaluating",model_name)
    
    test_outputs=[]
    print("Test data", end="\r")
    count=0
    for batch in datasets[size]["test"].iter(BS):
        output=model(torch.stack(batch["pixel_values"]).to(DEVICE))
        output=output.logits.detach().cpu().argmax(1) #remove gradient, take out of GPU, get highest value index
        test_outputs+=output.tolist()
        count+=len(batch["pixel_values"])
        print(f'Test data ({count/len(datasets[size]["test"])*100:.02f}%)', end="\r")

    train_outputs=[]
    print("Train data", end="\r")
    count=0
    for batch in datasets[size]["train"].iter(BS):
        output=model(torch.stack(batch["pixel_values"]).to(DEVICE))
        output=output.logits.detach().cpu().argmax(1)
        train_outputs+=output.tolist()
        count+=len(batch["pixel_values"])
        print(f'Train data ({count/len(datasets[size]["train"])*100:.02f}%)', end="\r")
    total_outputs=train_outputs+test_outputs

    y_pred[model_name]={}
    y_pred[model_name]["train"]=train_outputs
    y_pred[model_name]["test"]=test_outputs
    y_pred[model_name]["total"]=total_outputs
    
    model=None
    torch.cuda.empty_cache()
    gc.collect()

Evaluating mirluvams/swinv2-base-patch4-window16-256-popocatepetl
Evaluating mirluvams/swinv2-tiny-patch4-window16-256-popocatepetl
Evaluating mirluvams/beit-base-patch16-224-popocatepetl
Evaluating mirluvams/vit-base-patch16-224-popocatepetl


In [29]:
y_mean_train=np.unique(datasets[256]["train"]["class"],return_counts=True)[1].argmax() #class that repeats the most
n_train=len(datasets[256]["train"])
n_test=len(datasets[256]["test"])
y_pred["y_mean"]={}
y_pred["y_mean"]["train"]=list(np.full(n_train, y_mean_train))
y_pred["y_mean"]["test"]=list(np.full(n_test, y_mean_train))
y_pred["y_mean"]["total"]=list(np.full(n_train+n_test, y_mean_train))

In [30]:
y_median_train=np.median(datasets[256]["train"]["class"])
n_train=len(datasets[256]["train"])
n_test=len(datasets[256]["test"])
y_pred["y_median"]={}
y_pred["y_median"]["train"]=list(np.full(n_train, y_median_train))
y_pred["y_median"]["test"]=list(np.full(n_test, y_median_train))
y_pred["y_median"]["total"]=list(np.full(n_train+n_test, y_median_train))

In [31]:
#calculate standard metrics for classification predictors
def compute_metrics(y,y_p):
    return {
        "report":sklearn.metrics.classification_report(y,y_p, target_names=labels, output_dict=True, zero_division=0),
        "matrix":sklearn.metrics.confusion_matrix(y,y_p),
    }

In [68]:
metrics={}
for model_name, m_dict in y_pred.items():
    metrics[model_name]={}
    metrics[model_name]["train"]=compute_metrics(y_train, m_dict["train"])
    metrics[model_name]["test"]=compute_metrics(y_test, m_dict["test"])
    metrics[model_name]["total"]=compute_metrics(y_total, m_dict["total"])

In [69]:
print(metrics["mirluvams/swinv2-base-patch4-window16-256-popocatepetl"]["total"])

{'report': {'INACTIVE': {'precision': 0.9582319545823196, 'recall': 0.918026418026418, 'f1-score': 0.9376984126984127, 'support': 2574.0}, 'WITH_EXPLOSION': {'precision': 0.9210526315789473, 'recall': 0.48804780876494025, 'f1-score': 0.6380208333333334, 'support': 502.0}, 'WITH_FUME': {'precision': 0.9409870010353157, 'recall': 0.9775334608030593, 'f1-score': 0.9589121387960846, 'support': 8368.0}, 'WITH_FUME_AND_EXPLOSION': {'precision': 0.9570578231292517, 'recall': 0.9596418928520677, 'f1-score': 0.9583481160860001, 'support': 7037.0}, 'accuracy': 0.9491369514636654, 'macro avg': {'precision': 0.9443323525814585, 'recall': 0.8358123951116213, 'f1-score': 0.8732448752284576, 'support': 18481.0}, 'weighted avg': {'precision': 0.9489666467275358, 'recall': 0.9491369514636654, 'f1-score': 0.947026386178338, 'support': 18481.0}}, 'matrix': array([[2363,    2,  200,    9],
       [   5,  245,   67,  185],
       [  72,    7, 8180,  109],
       [  26,   12,  246, 6753]])}


In [70]:
#perf report | model slice i_p i_r i_f1 i_s ... fe_f1 fe_s acc m_p .. m_f1 w_p .. w_f1 

In [74]:
matrixes=[]
results=[]
for model_name, slices in metrics.items():
    for _slice, data in slices.items():
        matrixes.append(pd.Series({
            "MODEL":model_name,
            "SLICE":_slice,
            "MATRIX":data["matrix"].flatten().tolist()
        }))
        data=data["report"]
        r_data={
            "MODEL":model_name,
            "SLICE":_slice,
            "ACCURACY":data["accuracy"]
        }
        data["MACRO_AVG"]=data["macro avg"]
        data["WEIGHTED_AVG"]=data["weighted avg"]
        for label in ["INACTIVE", "WITH_EXPLOSION", "WITH_FUME", "WITH_FUME_AND_EXPLOSION", "MACRO_AVG", "WEIGHTED_AVG"]:
            r_data[label+"_PRECISION"]=data[label]["precision"]
            r_data[label+"_RECALL"]=data[label]["recall"]
            r_data[label+"_F1"]=data[label]["f1-score"]
            r_data[label+"_SUPPORT"]=data[label]["support"]
        results.append(pd.Series(r_data))
matrixes=pd.DataFrame(matrixes)
matrixes.to_feather("data/confusion_matrixes.feather")
results=pd.DataFrame(results)
results.to_feather("data/results.feather")

In [75]:
matrixes

,MODEL,SLICE,MATRIX
0,mirluvams/swinv2-base-patch4-window16-256-popo...,train,"[2164, 2, 142, 8, 4, 235, 51, 162, 46, 3, 7424..."
1,mirluvams/swinv2-base-patch4-window16-256-popo...,test,"[199, 0, 58, 1, 1, 10, 16, 23, 26, 4, 756, 51,..."
2,mirluvams/swinv2-base-patch4-window16-256-popo...,total,"[2363, 2, 200, 9, 5, 245, 67, 185, 72, 7, 8180..."
3,mirluvams/swinv2-tiny-patch4-window16-256-popo...,train,"[2023, 5, 276, 12, 10, 129, 73, 240, 428, 12, ..."
4,mirluvams/swinv2-tiny-patch4-window16-256-popo...,test,"[214, 0, 41, 3, 3, 17, 11, 19, 52, 3, 723, 59,..."
5,mirluvams/swinv2-tiny-patch4-window16-256-popo...,total,"[2237, 5, 317, 15, 13, 146, 84, 259, 480, 15, ..."
6,mirluvams/beit-base-patch16-224-popocatepetl,train,"[1929, 4, 374, 9, 9, 101, 83, 259, 230, 3, 695..."
7,mirluvams/beit-base-patch16-224-popocatepetl,test,"[189, 0, 67, 2, 3, 7, 12, 28, 28, 1, 747, 61, ..."
8,mirluvams/beit-base-patch16-224-popocatepetl,total,"[2118, 4, 441, 11, 12, 108, 95, 287, 258, 4, 7..."
9,mirluvams/vit-base-patch16-224-popocatepetl,train,"[2308, 3, 5, 0, 1, 443, 0, 8, 4, 7, 7511, 9, 2..."


In [76]:
results

,MODEL,SLICE,ACCURACY,INACTIVE_PRECISION,INACTIVE_RECALL,INACTIVE_F1,INACTIVE_SUPPORT,WITH_EXPLOSION_PRECISION,WITH_EXPLOSION_RECALL,WITH_EXPLOSION_F1,...,WITH_FUME_AND_EXPLOSION_F1,WITH_FUME_AND_EXPLOSION_SUPPORT,MACRO_AVG_PRECISION,MACRO_AVG_RECALL,MACRO_AVG_F1,MACRO_AVG_SUPPORT,WEIGHTED_AVG_PRECISION,WEIGHTED_AVG_RECALL,WEIGHTED_AVG_F1,WEIGHTED_AVG_SUPPORT
0,mirluvams/swinv2-base-patch4-window16-256-popo...,train,0.959235,0.967367,0.934370,0.950582,2316.0,0.943775,0.519912,0.670471,...,0.966120,6333.0,0.957168,0.852044,0.889123,16632.0,0.959168,0.959235,0.957369,16632.0
1,mirluvams/swinv2-base-patch4-window16-256-popo...,test,0.858302,0.868996,0.771318,0.817248,258.0,0.588235,0.200000,0.298507,...,0.887937,704.0,0.796016,0.689517,0.717791,1849.0,0.854669,0.858302,0.852869,1849.0
2,mirluvams/swinv2-base-patch4-window16-256-popo...,total,0.949137,0.958232,0.918026,0.937698,2574.0,0.921053,0.488048,0.638021,...,0.958348,7037.0,0.944332,0.835812,0.873245,18481.0,0.948967,0.949137,0.947026,18481.0
3,mirluvams/swinv2-tiny-patch4-window16-256-popo...,train,0.880712,0.805655,0.873489,0.838202,2316.0,0.701087,0.285398,0.405660,...,0.907697,6333.0,0.825085,0.740975,0.760771,16632.0,0.878601,0.880712,0.877053,16632.0
4,mirluvams/swinv2-tiny-patch4-window16-256-popo...,test,0.856679,0.786765,0.829457,0.807547,258.0,0.653846,0.340000,0.447368,...,0.890459,704.0,0.796850,0.732036,0.751907,1849.0,0.854458,0.856679,0.854140,1849.0
5,mirluvams/swinv2-tiny-patch4-window16-256-popo...,total,0.878307,0.803809,0.869075,0.835169,2574.0,0.695238,0.290837,0.410112,...,0.905979,7037.0,0.821986,0.740073,0.759961,18481.0,0.876152,0.878307,0.874771,18481.0
6,mirluvams/beit-base-patch16-224-popocatepetl,train,0.894000,0.877217,0.832902,0.854485,2316.0,0.789062,0.223451,0.348276,...,0.917342,6333.0,0.865696,0.727230,0.756712,16632.0,0.891784,0.894000,0.888326,16632.0
7,mirluvams/beit-base-patch16-224-popocatepetl,test,0.851812,0.851351,0.732558,0.787500,258.0,0.700000,0.140000,0.233333,...,0.885774,704.0,0.815264,0.665690,0.692423,1849.0,0.848790,0.851812,0.844148,1849.0
8,mirluvams/beit-base-patch16-224-popocatepetl,total,0.889779,0.874845,0.822844,0.848048,2574.0,0.782609,0.215139,0.337500,...,0.914181,7037.0,0.861281,0.721078,0.750512,18481.0,0.887511,0.889779,0.883935,18481.0
9,mirluvams/vit-base-patch16-224-popocatepetl,train,0.995070,0.996976,0.996546,0.996761,2316.0,0.915289,0.980088,0.946581,...,0.995094,6333.0,0.976828,0.991718,0.983995,16632.0,0.995228,0.995070,0.995117,16632.0


In [85]:
model_archives={
    "mirluvams/swinv2-base-patch4-window16-256-popocatepetl":"/home/miriam/tmp/models/swinv2-base/trainer_state.json",
    "mirluvams/swinv2-tiny-patch4-window16-256-popocatepetl":"/home/miriam/tmp/models/swinv2-tiny/trainer_state.json",
    "mirluvams/beit-base-patch16-224-popocatepetl":"/home/miriam/tmp/models/beit-base/trainer_state.json",
    "mirluvams/vit-base-patch16-224-popocatepetl":"/home/miriam/tmp/models/vit-base/trainer_state.json",
}

In [87]:
import json
data=[]
for model_name, model_archive in model_archives.items():
    with open(model_archive, "r") as f:
        d = json.load(f)["log_history"] #isolate logs
        tmp=None
        for step in d[:-1]: #skip conflicting line at end
            is_eval="eval_f1" in step.keys()
            if not is_eval:
                tmp={
                    "MODEL":model_name,
                    "EPOCH":step["epoch"],
                    "GRAD_NORM":step["grad_norm"],
                    "LR":step["learning_rate"],
                    "TRAIN_LOSS":step["loss"],
                    "STEP":step["step"],
                }
            else:
                tmp["EVAL_ACCURACY"]=step["eval_accuracy"]
                tmp["EVAL_F1"]=step["eval_f1"]
                tmp["EVAL_LOSS"]=step["eval_loss"]
                data.append(pd.Series(tmp))
                tmp=None

In [89]:
logs=pd.DataFrame(data)
logs

,MODEL,EPOCH,GRAD_NORM,LR,TRAIN_LOSS,STEP,EVAL_ACCURACY,EVAL_F1,EVAL_LOSS
0,mirluvams/swinv2-base-patch4-window16-256-popo...,1.0,3.367370e+05,0.000048,0.6279,693,0.835046,0.847354,0.454011
1,mirluvams/swinv2-base-patch4-window16-256-popo...,2.0,3.514949e+05,0.000046,0.4469,1386,0.839913,0.846267,0.445694
2,mirluvams/swinv2-base-patch4-window16-256-popo...,3.0,3.868218e+05,0.000044,0.4012,2079,0.857761,0.862673,0.407179
3,mirluvams/swinv2-base-patch4-window16-256-popo...,4.0,2.568965e+05,0.000042,0.3622,2772,0.851812,0.858742,0.443033
4,mirluvams/swinv2-base-patch4-window16-256-popo...,5.0,1.765064e+05,0.000040,0.3198,3465,0.857761,0.861967,0.420303
...,...,...,...,...,...,...,...,...,...
95,mirluvams/vit-base-patch16-224-popocatepetl,21.0,1.010245e-02,0.000008,0.0890,14553,0.852893,0.853526,2.384337
96,mirluvams/vit-base-patch16-224-popocatepetl,22.0,3.593377e-02,0.000006,0.0606,15246,0.850730,0.851742,2.464778
97,mirluvams/vit-base-patch16-224-popocatepetl,23.0,1.312766e+00,0.000004,0.0673,15939,0.856138,0.857009,2.413936
98,mirluvams/vit-base-patch16-224-popocatepetl,24.0,3.430031e+06,0.000002,0.0431,16632,0.852353,0.853692,2.414349


In [93]:
logs.to_feather("data/training_logs.feather")